In [1]:
def pdrf():
    global c_pdrf
    for i, demand in enumerate(d_pdrf[0]):
        u_pdrf[d_pdrf[1][i]] += demand * np.floor(rates[0][i] * cycleIterations)
        c_pdrf += demand * np.floor(rates[0][i] * cycleIterations)
        s_pdrf[d_pdrf[1][i]] += ds_pdrf[0][i] * np.floor(rates[0][i] * cycleIterations)

In [2]:
def drf():
    global c_drf
    while(1):
        i = np.argmin(s_drf)
        if (r < d_drf[i] + c_drf).any():
            break
        else:
            u_drf[i] += d_drf[i]
            s_drf[i] += ds_drf[0][i]
            c_drf += d_drf[i]

In [3]:
def update():
    i = ds_pdrf[0].argmax()

    d_pdrf[0] = np.delete(d_pdrf[0], i, axis=0)
    d_pdrf[1] = np.delete(d_pdrf[1], i, axis=0)
    ds_pdrf[0] = np.delete(ds_pdrf[0], i, axis=0)
    ds_pdrf[1] = np.delete(ds_pdrf[1], i, axis=0)

    s_max = np.max(ds_pdrf[0]) * np.ones(len(d_pdrf[0]))
    rates = [s_max / ds_pdrf[0], np.expand_dims(np.arange(len(d_pdrf[0]),), axis=1)]

    cycleDemands = np.sum(rates[0][:,np.newaxis] * d_pdrf[0], axis=0)
    cycleIterations = np.min((r - c_pdrf) / cycleDemands)

#    print(i)
#    print(cycleDemands)
#    print(cycleIterations)

In [4]:
def check_cycles():
    global cycleDemands
    global cycleIterations
    s_max = np.max(ds_pdrf[0]) * np.ones(len(d_pdrf[0]))
    rates = [s_max / ds_pdrf[0], np.expand_dims(np.arange(len(d_pdrf[0]),), axis=1)]

    cycleDemands = np.sum(rates[0][:,np.newaxis] * d_pdrf[0], axis=0)
    cycleIterations = np.min((r - c_pdrf) / cycleDemands)

    print(cycleDemands)
    print(cycleIterations)

In [5]:
import numpy as np
import math
from numpy import random

numberOfUsers = 10000
numberOfResources = 10

r_low = 1000000
r_high = 10000001

d_low = 1
d_high = 11

r = np.random.randint(r_low, r_high, numberOfResources) #resources
c = np.zeros(numberOfResources) #consumed amounts
#d = [np.random.randint(d_low, d_high, (numberOfUsers, numberOfResources)), np.expand_dims(np.arange(10), axis=1)] #demand vector
d = np.random.randint(d_low, d_high, (numberOfUsers, numberOfResources)) #demand vector

print(d)

[[ 8  9  6 ...  3  3  5]
 [ 6  1  4 ...  2 10  9]
 [ 6  1  3 ...  8  7  9]
 ...
 [ 8 10  4 ...  2  4  8]
 [ 7  6  1 ...  2  8  5]
 [ 5  7 10 ...  2  8  2]]


In [6]:
u_drf = np.zeros((numberOfUsers, numberOfResources)) #resources allocated to users
c_drf = np.zeros(numberOfResources) #resources allocated to users
s_drf = np.zeros(numberOfUsers) #allocated dominant shares of users
ds_drf = [(d / r).max(axis=1), np.expand_dims(np.arange(10), axis=1)]
d_drf = np.copy(d)

u_pdrf = np.zeros((numberOfUsers, numberOfResources)) #resources allocated to users
c_pdrf = np.zeros(numberOfResources) #resources allocated to users
s_pdrf = np.zeros(numberOfUsers) #allocated dominant shares of users
ds_pdrf = [(d / r).max(axis=1), np.expand_dims(np.arange(numberOfUsers), axis=1)]
d_pdrf = [d, np.expand_dims(np.arange(numberOfUsers), axis=1)]

s_max = np.max(ds_pdrf[0]) * np.ones(len(d_pdrf[0]))
rates = [s_max / ds_pdrf[0], np.expand_dims(np.arange(len(d_pdrf[0]),), axis=1)]

cycleDemands = np.sum(rates[0][:,np.newaxis] * d_pdrf[0], axis=0)
cycleIterations = np.min(r / cycleDemands)

In [7]:
print(d_pdrf[0])

[[ 8  9  6 ...  3  3  5]
 [ 6  1  4 ...  2 10  9]
 [ 6  1  3 ...  8  7  9]
 ...
 [ 8 10  4 ...  2  4  8]
 [ 7  6  1 ...  2  8  5]
 [ 5  7 10 ...  2  8  2]]


print(r)

In [8]:
drf()

In [9]:
pdrf()

In [10]:
difference = (u_drf - u_pdrf) / d
print(difference)
#for line in difference:
#    print(line)

[[1. 1. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [11]:
print("Number of Overallocations is:", np.count_nonzero(difference < 0, axis=0)[0])
print("Number of Underallocations is:", np.count_nonzero(difference, axis=0)[0] - np.count_nonzero(difference < 0, axis=0)[0])
print("Maximum Overallocation is", int(np.absolute(difference.min(axis=0).min())), "task(s).")
print("Maximum Underallocation is", int(difference.max(axis=0).max()), "task(s).")


Number of Overallocations is: 3
Number of Underallocations is: 4021
Maximum Overallocation is 1 task(s).
Maximum Underallocation is 1 task(s).


check_cycles()

for i in range(10):
    update()

print(len(d_pdrf[0]))

print(u_pdrf)

print(np.sort((u_drf / r).max(axis=1)))

print(np.sort((u_pdrf / r).max(axis=1)))